In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import os

In [2]:
# define paths
data_path = 'data'
output_path = 'output'

# set parent folder as working directory
os.chdir('..')
os.getcwd()

'c:\\Users\\isaldiviagonzatti\\Downloads\\gitFiles\\MScThesisCode\\FLP'

In [3]:
# load data 
optiData = pd.read_csv(os.path.join(output_path, 'optiData.csv'), index_col=0)

In [4]:
# define supply of PAL grouped by orig nodes. 
# Tonnes are the sum of all farms corresponding to node (note that mean is applied because of matrix form)
supplyPAL = pd.DataFrame(optiData.groupby('origs', as_index=False).agg(
                                            leavesTonne=('leavesTonne','mean'),
                                        )
                     )

In [5]:
# define number of PAL origs and facilities dests 
pal = list(set(optiData.origs))  # PAL points
fac = list(set(optiData.dests))  # candidate plant locations

In [6]:
optiData.head()

,origs,dests,path_length,osmidOrig,areaHa,leavesTonne,distance,totalDist
0,3610149725,2656380182,157664.026,3610149725,15.49063,1270.231645,319.129599,157.983156
1,3610149725,5294901777,83956.035,3610149725,15.49063,1270.231645,319.129599,84.275165
2,3610149725,1321571077,201954.500,3610149725,15.49063,1270.231645,319.129599,202.273630
3,3610149725,3677820110,75493.804,3610149725,15.49063,1270.231645,319.129599,75.812934
4,3610149725,3510580697,21814.095,3610149725,15.49063,1270.231645,319.129599,22.133225


In [7]:
# scenarios transport costs (tkm)
transCost1 = 0.20
transCost2 = 0.275
transCost3 = 0.4

In [8]:
# calculate transport cost: for each tonne transported, cost will be
# cost of transporting that tonne in a specific trip

optiData['costTrans'] = optiData.totalDist * transCost2 # * 1 tonne

In [9]:
# transport costs converted into dict of the form (dests, origs): cost for tonne per trip
costTrans = optiData.groupby(['dests','origs']).apply(lambda x: np.float64(x['costTrans'].values).round(2)).to_dict()

In [10]:
#scenarios X MM / 15 years + annual operating costs 350 M
costFac1 = 2_000_000/15 + 350_000
costFac2 = 3_000_000/15 + 350_000
costFac3 = 4_000_000/15 + 350_000

In [11]:
# The (annual) facility costs 
fac_cost = { i : costFac2 for i in fac }

# capacity of 170,428 tonnes/year
capacity = { i : 170428 for i in fac }

# supply of PAL from each (aggregated) farm
supply = dict(zip(supplyPAL.origs, np.int64(supplyPAL.leavesTonne)))

In [12]:
print('Example data (not related to each other):', 
      '\n PAL node:', pal[:1], 
      '\n Facility node:', fac[:1], 
      '\n Transport costs per tonne for specific trip:', next(iter( costTrans.items() )) ,
      '\n Facility costs:', next(iter( fac_cost.items() )) , 
      '\n Facility capacity:', next(iter( capacity.items() )) , 
      '\n Supply of PAL per node:', next(iter( supply.items() ))  
      ) 

Example data (not related to each other): 
 PAL node: [5060446209] 
 Facility node: [4950368263] 
 Transport costs per tonne for specific trip: ((183803630, 185483242), 23.29) 
 Facility costs: (4950368263, 550000.0) 
 Facility capacity: (4950368263, 170428) 
 Supply of PAL per node: (185483242, 1531)


In [13]:
# create model 
m = gp.Model()

# create variables, x[i,j] = number of units that PAL node j sends to facility i
x = m.addVars( fac, pal, vtype=GRB.INTEGER )

# create variables, y[i] = 1 if locate a facility at site i
y = m.addVars( fac, vtype=GRB.BINARY )

In [14]:
# Objective: minimize transportation cost (annual) + facility cost (annual)
m.setObjective( gp.quicksum( costTrans[i,j] * x[i,j] for i in fac for j in pal ) + gp.quicksum( fac_cost[i] * y[i] for i in fac) , GRB.MINIMIZE )

In [15]:
# Constraints: each palPoint point should send its full supply
m.addConstrs( gp.quicksum( x[i,j] for i in fac ) == supply[j] for j in pal )

# Constraints: if no factory is built at site i, it cannot receive any PAL
m.addConstrs( gp.quicksum( x[i,j] for j in pal ) <= capacity[i] * y[i] for i in fac )

# Optional strengthening constraints
m.addConstrs( x[i,j] <= supply[j] * y[i] for i in fac for j in pal )

m.update()

In [16]:
%%time

# force Gurobi to use concurrent method (primal simplex, dual simplex, barrier all at the same time!)
m.Params.Method = 3

m.setParam('MIPGap', 0.02)
m.setParam('Timelimit', 300)

# solve
m.optimize()

Set parameter Method to value 3
Set parameter MIPGap to value 0.02
Set parameter TimeLimit to value 300
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: AMD Ryzen 7 5700U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 560357 rows, 559205 columns and 2235365 nonzeros
Model fingerprint: 0xcb03057e
Variable types: 0 continuous, 559205 integer (485 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+05]
  Objective range  [6e-02, 6e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+01, 6e+04]
Found heuristic solution: objective 3.767337e+08
Presolve time: 3.08s
Presolved: 560357 rows, 559205 columns, 2235365 nonzeros
Variable types: 0 continuous, 559205 integer (485 binary)
Found heuristic solution: objective 3.767201e+08
Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Root barrier log...

Orderi

In [17]:
print("The total yearly cost of the optimal solution is $" , m.objVal)

The total yearly cost of the optimal solution is $ 29730963.27


In [18]:
# if binary is 1 for fac, fac is open. store in medians
medians = [ i for i in fac if y[i].x > 0.5 ]
print("In the optimal solution, the number of opened biogas plants are: ", len(medians))

In the optimal solution, the number of opened biogas plants are:  28


In [19]:
# for each fac open, retrieve farms that send PAL to fac and the quantity sent
k = len(medians)
assignedL = []
facL = []
quantityL = []

for p in range(k):
    i = medians[p] 
    for j in pal:
        if x[i,j].x > 0.5: 
            assigned = j
            quantity = x[i,j].x
            facR = i  
            assignedL.append(assigned)
            facL.append(facR)
            quantityL.append(quantity)

In [20]:
# store in dataframe
results = pd.DataFrame(list(zip(facL,assignedL,quantityL)), columns=['facility', 'nodePAL', 'quantity'])
results.head()

,facility,nodePAL,quantity
0,2683369498,1623183425,20762.0
1,2683369498,3279845497,968.0
2,2683369498,5260581064,965.0
3,2683369498,1088156051,25024.0
4,2683369498,3279862178,1782.0


In [21]:
results.to_csv(os.path.join(output_path,'resultsOpti.csv'), index=False)

In [22]:
# for each fac open, retrieve farms that send PAL to fac. 
# k = len(medians)
# assignedL = []
# for p in range(k):
#     i = medians[p] 
#     assigned = [ j for j in pal if x[i,j].x > 0.5 ]
#     assignedL.append(assigned)